In [192]:
owner = 'test-18'

In [193]:
import polaris as po
import datamol as dm
import numpy as np

In [185]:
#benchmark = po.load_benchmark("polaris/pkis1-kit-wt-mut-c-1")
#train, test = benchmark.get_train_test_split(featurization_fn=dm.to_fp)
#train, test = benchmark.get_train_test_split()
# ys = train.y
# ys = np.stack([ys[target] for target in benchmark.target_cols], axis=1)
# mask = ~np.any(np.isnan(ys), axis=1)
# mask.sum()

# X_train = train.X[mask]
# y_train = ys[mask]



In [195]:
## Balancing multi-task

from functools import partial

from imblearn.over_sampling import SMOTE
import polaris as po
import datamol as dm
import numpy as np

mapping = {
 (0.0, 0.0, 0.0): 0,
 (1.0, 0.0, 0.0): 1,
 (1.0, 0.0, 1.0): 2,
 (1.0, 1.0, 0.0): 3,
 (1.0, 1.0, 1.0): 4,
}
inv_mapping = {v: k for k, v in mapping.items()}

# load dataset
benchmark = po.load_benchmark("polaris/pkis1-kit-wt-mut-c-1")
# use ECFP fingerprint
train, test = benchmark.get_train_test_split(featurization_fn=partial(dm.to_fp, fp_type='ecfp'))

# define order of target values
target_order = ['CLASS_KIT', 'CLASS_KIT_(T6701_mutant)', 'CLASS_KIT_(V560G_mutant)']

# reshape the y values for convenience
ys = train.y
ys = np.stack([ys[target] for target in target_order], axis=1)
ys.shape

# remove the rows with NaN values
mask = ~np.any(np.isnan(ys), axis=1)
mask.sum()
X = train.X[mask]
ys = ys[mask]

ys_scalarized = [tuple(item) for item in ys]
ys_scalarized = [mapping[item] for i, item in enumerate(ys_scalarized)]

X_resampled, y_resampled = SMOTE(k_neighbors=2).fit_resample(X, ys_scalarized)
y_resampled = [inv_mapping[item] for i, item in enumerate(y_resampled)]


X_train = X_resampled
y_train = y_resampled

2024-06-21 16:33:52.172 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).
2024-06-21 16:33:52.176 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).


In [196]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from bayes_opt import BayesianOptimization


def get_param(optimizer):
    best_params = optimizer.max['params']
    best_params['n_estimators'] = int(best_params['n_estimators'])
    best_params['max_depth'] = int(best_params['max_depth'])
    best_params['min_samples_split'] = int(best_params['min_samples_split'])
    best_params['min_samples_leaf'] = int(best_params['min_samples_leaf'])
    best_params['bootstrap'] = bool(best_params['bootstrap'])
    return best_params

In [197]:
def opt_multi_targets_pca(param_bounds):

    X_train = X_resampled
    y_train = y_resampled
    # X_train = train.X[mask]
    # y_train = ys[mask]

    def objective(input_dim, n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features, bootstrap):

        
        rf = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            bootstrap=bool(bootstrap),
            random_state=42
        )

        pipeline = Pipeline([
            ('pca', PCA(int(input_dim))),
            ('scaler', StandardScaler()),
            ('rf', rf)
        ])
        
        # Use K-Fold cross-validation
        kfold = KFold(n_splits=2, shuffle=True, random_state=42)
        
        # Evaluate the model using cross-validation
        scoring = make_scorer(average_precision_score, needs_proba=True)
        
        cv_scores = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring=scoring)
        return cv_scores.mean()

    
    


    
    def rf_multi_cv(n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features, bootstrap):
        # Convert parameters to int where necessary
        n_estimators = int(n_estimators)
        max_depth = int(max_depth)
        min_samples_split = int(min_samples_split)
        min_samples_leaf = int(min_samples_leaf)
        max_features = max(min(max_features, 0.999), 1e-3)  # to avoid 0 and 1

        


        
        rf = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            bootstrap=bool(bootstrap),
            random_state=42
        )
        
        # Perform cross-validation
        acc = 0
        for i in range(5):
            cval = cross_val_score(rf, X_train, y_train, scoring='accuracy', cv=2)
            acc += cval.mean()
        
        return acc/5
    
    optimizer = BayesianOptimization(
        f=rf_multi_cv,
        pbounds=param_bounds,
        random_state=42,
        verbose=2
    )
    
    optimizer.maximize(init_points=10, n_iter=50)

        
    best_params = get_param(optimizer)
    
    print("Best parameters found: ", best_params)
    
    best_rf = RandomForestClassifier(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        min_samples_split=best_params['min_samples_split'],
        min_samples_leaf=best_params['min_samples_leaf'],
        max_features=best_params['max_features'],
        bootstrap=best_params['bootstrap'],
        random_state=42
    )
    
    best_rf.fit(X_train, y_train)
    y_pred = best_rf.predict(test.X)
    #cval = cross_val_score(best_rf, X_train, y_train, scoring='accuracy', cv=3)
    
    accuracy = cval.mean()
    print(f'Accuracy: {accuracy:.4f}')

    return best_rf, best_params


In [198]:
def opt_multi_targets(param_bounds):

    X_train = X_resampled
    y_train = y_resampled
    # X_train = train.X[mask]
    # y_train = ys[mask]

    def rf_multi_cv(n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features, bootstrap):
        # Convert parameters to int where necessary
        n_estimators = int(n_estimators)
        max_depth = int(max_depth)
        min_samples_split = int(min_samples_split)
        min_samples_leaf = int(min_samples_leaf)
        max_features = max(min(max_features, 0.999), 1e-3)  # to avoid 0 and 1

        


        
        rf = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            bootstrap=bool(bootstrap),
            random_state=42
        )
        
        # Perform cross-validation
        acc = 0
        for i in range(5):
            cval = cross_val_score(rf, X_train, y_train, scoring='accuracy', cv=2)
            acc += cval.mean()
        
        return acc/5
    
    optimizer = BayesianOptimization(
        f=rf_multi_cv,
        pbounds=param_bounds,
        random_state=42,
        verbose=2
    )
    
    optimizer.maximize(init_points=10, n_iter=50)

        
    best_params = get_param(optimizer)
    
    print("Best parameters found: ", best_params)
    
    best_rf = RandomForestClassifier(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        min_samples_split=best_params['min_samples_split'],
        min_samples_leaf=best_params['min_samples_leaf'],
        max_features=best_params['max_features'],
        bootstrap=best_params['bootstrap'],
        random_state=42
    )
    
    best_rf.fit(X_train, y_train)
    y_pred = best_rf.predict(test.X)
    #cval = cross_val_score(best_rf, X_train, y_train, scoring='accuracy', cv=3)
    
    accuracy = cval.mean()
    print(f'Accuracy: {accuracy:.4f}')

    return best_rf, best_params


In [191]:
param_bounds = {
    'n_estimators': (10, 200),
    'max_depth': (1, 50),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 10),
    'max_features': (0.1, 0.999),
    'bootstrap': (0, 1)  # Treated as a boolean
}

best_rf, best_params = opt_multi_targets(param_bounds = param_bounds)


|   iter    |  target   | bootstrap | max_depth | max_fe... | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 0.2057    | 0.3745    | 47.59     | 0.7581    | 6.388     | 3.248     | 39.64     |
| 2         | 0.2057    | 0.05808   | 43.44     | 0.6404    | 7.373     | 2.165     | 194.3     |
| 3         | 0.2935    | 0.8324    | 11.4      | 0.2635    | 2.651     | 4.434     | 109.7     |
| 4         | 0.2935    | 0.4319    | 15.27     | 0.6501    | 2.255     | 4.337     | 79.61     |
| 5         | 0.2057    | 0.4561    | 39.47     | 0.2795    | 5.628     | 6.739     | 18.83     |


KeyboardInterrupt: 

In [191]:
param_bounds = {
    'n_estimators': (10, 200),
    'max_depth': (1, 50),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 10),
    'max_features': (0.1, 0.999),
    'bootstrap': (0, 1)  # Treated as a boolean
}

best_rf, best_params = opt_multi_targets(param_bounds = param_bounds)


|   iter    |  target   | bootstrap | max_depth | max_fe... | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 0.2057    | 0.3745    | 47.59     | 0.7581    | 6.388     | 3.248     | 39.64     |
| 2         | 0.2057    | 0.05808   | 43.44     | 0.6404    | 7.373     | 2.165     | 194.3     |
| 3         | 0.2935    | 0.8324    | 11.4      | 0.2635    | 2.651     | 4.434     | 109.7     |
| 4         | 0.2935    | 0.4319    | 15.27     | 0.6501    | 2.255     | 4.337     | 79.61     |
| 5         | 0.2057    | 0.4561    | 39.47     | 0.2795    | 5.628     | 6.739     | 18.83     |


KeyboardInterrupt: 

In [182]:
y_pred = best_rf.predict(test.X)
#y_pred.shape

y_prob = best_rf.predict_proba(test.X)
y_prob = np.stack(y_prob, axis=1)
#y_prob.shape

y_pred_multi = {k: y_pred[:, idx] for idx, k in enumerate(benchmark.target_cols)}
y_prob_multi = {k: y_prob[:, idx, 1] for idx, k in enumerate(benchmark.target_cols)}

results_multi = benchmark.evaluate(y_pred=y_pred_multi, y_prob=y_prob_multi)


results_multi.name = "rf_multi_augm"
results_multi.description = best_params
results_multi.to_json('rf_multi_augm.json')

/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/pydantic/main.py:347: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `dict` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [160]:
results_multi

{
  "name": "rf_multi",
  "description": {
    "CLASS_KIT_(T6701_mutant)": {
      "bootstrap": true,
      "max_depth": 47,
      "max_features": 0.7580625536884532,
      "min_samples_leaf": 6,
      "min_samples_split": 3,
      "n_estimators": 39
    },
    "CLASS_KIT_(V560G_mutant)": {
      "bootstrap": true,
      "max_depth": 47,
      "max_features": 0.7580625536884532,
      "min_samples_leaf": 6,
      "min_samples_split": 3,
      "n_estimators": 39
    },
    "CLASS_KIT": {
      "bootstrap": true,
      "max_depth": 35,
      "max_features": 0.7658783536012476,
      "min_samples_leaf": 2,
      "min_samples_split": 7,
      "n_estimators": 178
    }
  },
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "dev",
  "benchmark_name": "pkis1-kit-wt-mut-c-1",
  "benchmark_owner": {
    "slug": "polaris",
    "external_id": "org_2gtoaJIVrgRqiIR8Qm5BnpFCbxu",
    "type": "organization"
  },
  "github_url": null,
  "paper_url": null,
  "contributors": null,
  "artifact_id": null,
  "benchmark_artifact_id": "polaris/pkis1-kit-wt-mut-c-1",
  "results": [
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "accuracy",
      "Score": 0.8390804598
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "accuracy",
      "Score": 0.8620689655
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "accuracy",
      "Score": 0.6781609195
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "f1",
      "Score": 0.3333333333
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "roc_auc",
      "Score": 0.7627201566
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "roc_auc",
      "Score": 0.7522222222
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "roc_auc",
      "Score": 0.730359147
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "pr_auc",
      "Score": 0.6407846394
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "pr_auc",
      "Score": 0.453787598
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "pr_auc",
      "Score": 0.659833638
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "mcc",
      "Score": 0.2789537398
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "cohen_kappa",
      "Score": 0.2039215686
    }
  ]
}

In [150]:
def opt_target(target, param_bounds):
    if isinstance(target, int) is True:
        target = benchmark.target_cols[target]
    
    ys = train.y[target]
    y_train = ys[mask]
    
    def rf_cv(n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features, bootstrap):
        # Convert parameters to int where necessary
        n_estimators = int(n_estimators)
        max_depth = int(max_depth)
        min_samples_split = int(min_samples_split)
        min_samples_leaf = int(min_samples_leaf)
        max_features = max(min(max_features, 0.999), 1e-3)  # to avoid 0 and 1
    
        rf = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            bootstrap=bool(bootstrap),
            random_state=42
        )
        
        # Perform cross-validation
        cval = cross_val_score(rf, X_train, y_train, scoring='accuracy', cv=3)
        
        return cval.mean()
    
    
    
    optimizer = BayesianOptimization(
        f=rf_cv,
        pbounds=param_bounds,
        random_state=42,
        verbose=2
    )
    optimizer.maximize(init_points=10, n_iter=30)
    best_params = get_param(optimizer)

    best_rf = RandomForestClassifier(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        min_samples_split=best_params['min_samples_split'],
        min_samples_leaf=best_params['min_samples_leaf'],
        max_features=best_params['max_features'],
        bootstrap=best_params['bootstrap'],
        random_state=42
    )
    
    best_rf.fit(X_train, y_train)

    return best_rf, best_params


In [151]:
param_bounds = {
    'n_estimators': (10, 200),
    'max_depth': (1, 50),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 10),
    'max_features': (0.1, 0.999),
    'bootstrap': (0, 1)  # Treated as a boolean
}

models = {target: opt_target(target, param_bounds = param_bounds) for target in benchmark.target_cols}

|   iter    |  target   | bootstrap | max_depth | max_fe... | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 0.9746    | 0.3745    | 47.59     | 0.7581    | 6.388     | 3.248     | 39.64     |
| 2         | 0.9746    | 0.05808   | 43.44     | 0.6404    | 7.373     | 2.165     | 194.3     |
| 3         | 0.9746    | 0.8324    | 11.4      | 0.2635    | 2.651     | 4.434     | 109.7     |
| 4         | 0.9746    | 0.4319    | 15.27     | 0.6501    | 2.255     | 4.337     | 79.61     |
| 5         | 0.9746    | 0.4561    | 39.47     | 0.2795    | 5.628     | 6.739     | 18.83     |
| 6         | 0.9746    | 0.6075    | 9.356     | 0.1585    | 9.54      | 9.725     | 163.6     |
| 7         | 0.9746    | 0.3046    | 5.786     | 0.7151    | 4.961     | 2.976     | 104.1     |
| 8         | 0.9746    | 0.03439   | 45.56     | 0.3326    | 6.963     | 4.494     | 108.8     |
| 9         | 0.9746

In [158]:
y_prob_ind = {target: model.predict_proba(test.X)[:, 1] for target, (model, params) in models.items()}
y_pred_ind = {target: model.predict(test.X) for target, (model, params) in models.items()}

best_params_ind = {target: params for target, (model, params) in models.items()}

results_ind = benchmark.evaluate(y_pred=y_pred_ind, y_prob=y_prob_ind)

results_ind.name = "rf_ind"
results_ind.description = best_params_ind
results_ind.to_json('rf_ind.json')

/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/pydantic/main.py:347: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `dict` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [159]:
results_ind

{
  "name": "rf_ind",
  "description": {
    "CLASS_KIT_(T6701_mutant)": {
      "bootstrap": true,
      "max_depth": 47,
      "max_features": 0.7580625536884532,
      "min_samples_leaf": 6,
      "min_samples_split": 3,
      "n_estimators": 39
    },
    "CLASS_KIT_(V560G_mutant)": {
      "bootstrap": true,
      "max_depth": 47,
      "max_features": 0.7580625536884532,
      "min_samples_leaf": 6,
      "min_samples_split": 3,
      "n_estimators": 39
    },
    "CLASS_KIT": {
      "bootstrap": true,
      "max_depth": 35,
      "max_features": 0.7658783536012476,
      "min_samples_leaf": 2,
      "min_samples_split": 7,
      "n_estimators": 178
    }
  },
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "dev",
  "benchmark_name": "pkis1-kit-wt-mut-c-1",
  "benchmark_owner": {
    "slug": "polaris",
    "external_id": "org_2gtoaJIVrgRqiIR8Qm5BnpFCbxu",
    "type": "organization"
  },
  "github_url": null,
  "paper_url": null,
  "contributors": null,
  "artifact_id": null,
  "benchmark_artifact_id": "polaris/pkis1-kit-wt-mut-c-1",
  "results": [
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "accuracy",
      "Score": 0.8390804598
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "accuracy",
      "Score": 0.8620689655
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "accuracy",
      "Score": 0.6551724138
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "f1",
      "Score": 0.2105263158
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "roc_auc",
      "Score": 0.7353228963
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "roc_auc",
      "Score": 0.7227777778
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "roc_auc",
      "Score": 0.8010662177
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "pr_auc",
      "Score": 0.6131561014
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "pr_auc",
      "Score": 0.4876573618
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "pr_auc",
      "Score": 0.6506629159
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "mcc",
      "Score": 0.2140944019
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "cohen_kappa",
      "Score": 0.122983871
    }
  ]
}